
## Zadania do wykonania

### Zadanie 1

Napisz dekorator, który autoryzuje użytkownika na podstawie podanego loginu i hasła. Do przechowywania danych należy wykorzystać klasę `shelve`. Do maskowania hasła można użyć następującej funkcji:

In [36]:
import hashlib, binascii
from os import urandom

def hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'),
                                salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')

hash_password('test')

'82fb8b5b78df658498f0a67e88db93a3a5b09788396e956a434ed34b9ff265b1fbe61093d3e567ecb490ea6ec8655b8d23e096dba46665026a5d039d785494d8196c142236a94d0a5e31d7f417eb1081d2310fa88789f6792a57ece2ce48ca5c'

Należy pamiętać, że `salt` należy utworzyć raz i zachować w chronionym pliku.

### Zadanie 2

Napisz dekorator, który ogranicza argument funkcji do zadanego przedziału.

### Zadanie 3

Napisz generator, który dla zadanego wielomianu i przedziału wartości, zwróci jego miejsca zerowe. Dla pierwszego wywołania wynikiem ma być pierwsze miejsce zerowe, dla drugiego, drugie itd. Można zastosować naiwny algorytm sprawdzający kolejne wartości wielomianu. Można również zastosować kodowanie wielomianu bezpośrednio w kodzie języka Python: `x**n*x**(n-1)`.

In [104]:
# zadanie 1


import shelve
import hashlib, binascii
from os import urandom
import functools
import getpass

def openShelve(name):
    return shelve.open(name)
def closeShelve(name2):
    name2.close()
    
def getSalt():
    salt = hashlib.sha256(urandom(60)).hexdigest().encode('ascii')  
    return salt
def generateSalt():
    shelf = openShelve("salt")
    try:
        salt2 = shelf['salt']
        print('error')
        closeShelve(shelf)
    except Exception as e:
        shelf['salt'] = getSalt()
        closeShelve(shelf)
        
def hash_password(password):
    shelf = openShelve("salt")
    salt = shelf['salt']
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'),
                                salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')
def addUser(login,haslo):
    shelf = openShelve("data")
    try:
        log = shelf[login]
        print('The is has already created')
        closeShelve(shelf)
    except Exception as e:
        shelf[login] = hash_password(haslo)
        closeShelve(shelf)
    
    
def call_safe(func):
    def wrapper(*args, **kwargs):
        
        x = 0
        for arg in args:
            if(x == 0):
                login = arg
            if(x == 1):
                haslo = arg
            x += 1
            
        shelf = openShelve("salt")
        shelf2 = openShelve("data")
        
        try:
            salt = shelf['salt']
            password = shelf2[login]
            
            pwdhash = hashlib.pbkdf2_hmac('sha512', haslo.encode('utf-8'),
                                salt, 100000)
            pwdhash = binascii.hexlify(pwdhash)
            
            passWd = (salt + pwdhash).decode('ascii')
            
            closeShelve(shelf)
            closeShelve(shelf2)
            
            if(password == passWd):
                return func(1,1)
            else:
                return func(0,0)
            
        except Exception as e:
            print('There is no user with this login in system!')

        return None

    return wrapper

@call_safe
def base_func(login,haslo):
    if(login & haslo):
        print('You are logged in system!')
    else:
        print('Uncorrect login data!!!')


# generateSalt()

# addUser('user1','passw1')

x = input('Enter youre login: ')
print(f'Enter youre password: ')    
y = getpass.getpass()
base_func(x,y)



Enter youre login: user1
Enter youre password: 
········
You are logged in system!


In [6]:
# zadanie 2

import functools

def call_safe(func):
    def wrapper(*args, **kwargs):
        try:
            for arg in args:
                if(not(arg>=0)):
                    print(f'\tElement is below range')
                    return func(0)
                if(not(arg<=100)):
                    print(f'\tElement is above range')
                    return func(100)
                else: 
                    return func(*args, **kwargs)
        except Exception as e:
            print(f'error')

        return None

    return wrapper

@call_safe
def base_func(p):
    print(f'\tWe wrote {p}')
    list1.append(p)


list1 = []
try:
    x = input('Amount of elements: ')
    for i in range(int(x)):
        y = input(f'Element {i+1}: ')
        base_func(float(y))
except Exception as e:
    print('NaN')
    
print(f'Elements {list1}')



Amount of elements: 3
Element 1: 10000.1212121
	Element is above range
	We wrote 100
Element 2: 00
	We wrote 0.0
Element 3: -1212102.1973619873
	Element is below range
	We wrote 0
Elements [100, 0.0, 0]


In [95]:
# zadanie 3

import math

n = 4
    
def func():
    x = -1000.00
    while x < 1000:
        c = eval('(x-2)*(x+31)*(6*x)*(x**2+2*x)')
        if c==0:
            yield x
        x += 0.1
        x = round(x,2)

mygenerator = func()   

g = 0
for i in mygenerator:
    print(f'{g+1}: {i}')
    if g == n: break
    g += 1
    

1: -31.0
2: -2.0
3: 0.0
4: 2.0
